In [ ]:
from ai_summarize import RevitAi


text: str = """
Um computador é uma máquina eletrônica que processa dados, realiza cálculos e executa tarefas complexas rapidamente. Ele é composto por hardware (componentes físicos) e software (programas) que, juntos, permitem manipulação de informações e automação de processos.
"""

tests = RevitAi()
results = tests.test_summarize(
    text
)

results

In [1]:
from ai_summarize import RevitAi


data: dict = {
  "text": "League of Legends é um jogo online de estratégia e ação desenvolvido pela Riot Games, onde duas equipes de cinco jogadores competem para destruir a base adversária. Cada jogador controla um 'campeão' com habilidades únicas e trabalha em equipe para ganhar vantagens no mapa, derrotar oponentes e conquistar objetivos estratégicos. Popular no cenário de eSports, o jogo é conhecido por sua complexidade e alta competitividade."
}

text: str = data.get("text")

tests = RevitAi()
results = tests.summarize(
    text
)

results

/home/joao-ubuntu/codes/revit/back/src/functions/ai_summarize.py:11: SyntaxWarning: invalid escape sequence '\s'
  WHITESPACE_HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip()))
/home/joao-ubuntu/codes/revit/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-11-10 18:51:52.736775: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-10 18:51:52.746871: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731275512.760463 

'O jogo League of Legends é um dos mais populares do mundo.'